# 0. Short description 

In the first part of the lab you are going to implement a standard document processing pipeline and then build a simple search engine based on it:
- starting from crawling documents, 
- then building an inverted index,
- and answering queries using this index.

Second part is devoted to spellchecking. You will implement some functions related to spellchecking.
- K-gram index
- weighted editorial distance
- Norvig spellchecker


# 1. [60] Building inverted index and answering queries

## 1.1. [10] Preprocessing

First, we need a unified approach to documents preprocessing. Implement a class responsible for that. Complete the code for given functions (most of them are just one-liners) and make sure you pass the tests. Make use of `nltk` library or any other you know.

In [1]:
import nltk

class Preprocessor:
    
    def __init__(self):
        self.stop_words = {'a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its',
                      'of', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with'}
        self.ps = nltk.stem.PorterStemmer()

    
    def tokenize(self, text):
        #TODO word tokenize text using nltk lib
        return nltk.tokenize.word_tokenize(text)        
        
    def stem(self, word, stemmer):
        #TODO stem word using provided stemmer
        return stemmer.stem(word)

    
    def is_apt_word(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        return word.isalpha() and word not in self.stop_words

    def is_apt_word_without_stop_words(self, word):
        #TODO check if word is appropriate - not a stop word and isalpha, 
        # i.e consists of letters, not punctuation, numbers, dates
        return word.isalpha()
    
    def preprocess(self, text):
        #TODO combine all previous methods together: tokenize lowercased text 
        # and stem it, ignoring not appropriate words
        words = self.tokenize(text.lower())
        words = [self.stem(word, self.ps) for word in words]
        words = list(filter(self.is_apt_word, words))
        return words

### 1.1.1. Tests

In [2]:
prep = Preprocessor()
text = 'To be, or not to be, that is the question'

assert prep.tokenize(text) == ['To', 'be', ',', 'or', 'not', 'to', 'be', ',', 'that', 'is', 'the', 'question']
assert prep.stem('retrieval', prep.ps) == 'retriev'
assert prep.is_apt_word('qwerty123') is False
assert prep.preprocess(text) == ['or', 'not', 'question']

## 1.2. [35] Crawling and Indexing

### 1.2.1. [10] Base classes

Here are some base classes you will need for writing an indexer. The code from the first lab's solution is given, still you need to change some of it, namely, the `parse` function (it is also possible to use your own implementation from the first homework, but make sure that it is correct). The reason is it always makes complete parsing, which we want to avoid when we only need links, for example, or a specific portion of text.

In [3]:
import requests
from urllib.parse import quote
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.parse
import os


class Document:

    def __init__(self, url):
        self.url = url
    
    def download(self):
        try:
            response = requests.get(self.url)
            if response.status_code == 200:
                self.content = response.content
                return True
            else:
                return False
        except:
            return False

    def persist(self, path):
        with open(os.path.join(path, quote(self.url).replace('/', '_')), 'wb') as f:
            f.write(self.content)


class HtmlDocument(Document):

    def normalize(self, href):
        if href is not None and href[:4] != 'http':
            href = urllib.parse.urljoin(self.url, href)
        #else:
        return href

    def parse(self, anchors=True, images=False, text=False, article=False):
        #TODO change this method
        
        def tag_visible(element):
            if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
                return False
            if isinstance(element, Comment):
                return False
            return True
        
        if self.download():
            model = BeautifulSoup(self.content)
            

            self.anchors = []
            if anchors:
                a = model.find_all('a')
                for anchor in a:
                    href = self.normalize(anchor.get('href'))
                    text = anchor.text
                    if href != None:
                        self.anchors.append((text, href))

            self.images = []
            if images:
                i = model.find_all('img')
                for img in i:
                    href = self.normalize(img.get('src'))
                    self.images.append(href)

            self.text = []
            if text:
                texts = model.findAll(text=True)
                visible_texts = list(filter(tag_visible, texts))
                self.text = u" ".join(t.strip() for t in visible_texts)
            
            # method for retrieving the text only from the article itself
            if article:
                visible_texts = ""
                if model.find('p', class_='Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x'):
                    paragraphs = model.find_all('p', class_='Paragraph-paragraph-2Bgue ArticleBody-para-TD_9x')
                    for paragraph in paragraphs:
                        text = paragraph.getText()
                        visible_texts += text.strip() 
                self.text = visible_texts

### 1.2.2. [20] Main class

The main indexer logic is here. We organize it as a crawler generator that adds certain visited pages to inverted index and saves them on disk. 

- `crawl_generator_for_index` method crawles the given website doing BFS, starting from `source` within given `depth`. Considers only inner pages (of a form https://www.reuters.com/...) for visiting. To speed up, doesn't consider for visiting pages with content type other than html: '.pdf', '.mp3', '.avi', '.mp4', '.txt'. If encounters an article page (of a form https://www.reuters.com/article/...), saves its content in a file in `collection_path` folder and populates the inverted index calling `index_doc` method. When done, saves on disk three resulting dictionaries:
    - `doc_urls`, `doc_id:url`
    - `index`, `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    - `doc_lengths`, `doc_id:doc_length` 


    `limit` parameter is given for testing - if not `None`, break the loop when number of saved articles exceeds the `limit` and return without writing dictionaries to disk.
    
    
- `index_doc` method parses and preprocesses the content of a `doc` and adds it to the inverted index. Also keeps track of document lengths in a `doc_lengths` dictionary.

Your crawler have to print visited urls as it runs.

In [7]:
from collections import Counter
from queue import Queue
import pickle
import os

class Indexer:

    def __init__(self):      
        # dictionaries to populate
        self.doc_urls = {}        
        self.index = {}
        self.doc_lengths = {}
        # preprocessor
        self.prep = Preprocessor()
        
    
    def crawl_generator_for_index(self, source, depth, collection_path="collection", limit=None):        
        #TODO generate url-s for visiting
        
        if not os.path.exists(collection_path):
            os.makedirs(collection_path)

        article_counter = 0
        ignore_extension = ['pdf', 'mp3', 'avi', 'mp4', 'txt']
        
        self.doc_data = HtmlDocument(source)
        self.doc_data.parse(images=False)
        
        processed = set()
        processed.add(source)
        
        q = Queue()
        for anchor in self.doc_data.anchors:
            
            if anchor[1].startswith("https://www.reuters.com") and anchor[1].split(".")[-1] not in ignore_extension:

                q.put([anchor[1], 1])

        yield self.doc_data
        
        while not q.empty() and (limit == None or article_counter < limit):
            new_url = q.get()
                
            if new_url[0] not in processed and new_url[1] < depth:
    
                try:
                    self.doc_data = HtmlDocument(new_url[0])
                
                    if len(new_url[0].split("/")) > 3 and new_url[0].split("/")[3] == "article":
                        self.doc_data.parse(article=True)
                        self.doc_data.persist(collection_path)
                        self.index_doc(self.doc_data, len(self.doc_urls))
                        article_counter += 1
                    else:
                        self.doc_data.parse()
                    

                    for anchor in self.doc_data.anchors:
                        
                        if anchor[1] not in processed and anchor[1].startswith("https://www.reuters.com") \
                        and anchor[1].split(".")[-1] not in ignore_extension:
                            q.put([anchor[1], new_url[1] + 1])
                    
                    processed.add(new_url[0])
                    yield self.doc_data
                    
                except Exception as e:
                    processed.add(new_url[0])

        # save results for later use
        with open('doc_urls.p', 'wb') as fp:
            pickle.dump(self.doc_urls, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('inverted_index.p', 'wb') as fp:
            pickle.dump(self.index, fp, protocol=pickle.HIGHEST_PROTOCOL)
        with open('doc_lengths.p', 'wb') as fp:
            pickle.dump(self.doc_lengths, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
        
    def index_doc(self, doc, doc_id):
        self.doc_urls[doc_id] = doc.url
        self.doc_lengths[doc_id] = len(self.prep.preprocess(doc.text))
        
        for term in self.prep.preprocess(doc.text):
            if term not in self.index:
                self.index[term] = [1, (doc_id, 1)]
            else:
                self.index[term][0] += 1
                doc_ids = list(map(lambda x: x[0], self.index[term][1:]))

                if doc_id in doc_ids:
                    idx = doc_ids.index(doc_id)
                    freq = self.index[term][idx+1][1]
                    self.index[term][idx+1] = (doc_id, freq+1)
                else:
                    self.index[term].append((doc_id, 1))

### 1.2.3. Tests

Please make sure your crawler prints out urls with `print(k, c.url)`

In [8]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/technology", 2, "test_collection", 5):
    print(k, c.url)
    k += 1

assert type(indexer.index) is dict
assert type(indexer.index['reuter']) is list
assert type(indexer.index['reuter'][0]) is int
assert type(indexer.index['reuter'][1]) is tuple

1 https://www.reuters.com/technology
2 https://www.reuters.com/
3 https://www.reuters.com/home
4 https://www.reuters.com/world
5 https://www.reuters.com/world/us-politics
6 https://www.reuters.com/world/us
7 https://www.reuters.com/world/uk
8 https://www.reuters.com/world/china
9 https://www.reuters.com/world/india
10 https://www.reuters.com/world/americas
11 https://www.reuters.com/world/asia-pacific
12 https://www.reuters.com/world/europe
13 https://www.reuters.com/world/middle-east-africa
14 https://www.reuters.com/world/the-great-reboot
15 https://www.reuters.com/business
16 https://www.reuters.com/business/sustainable-business
17 https://www.reuters.com/business/legal
18 https://www.reuters.com/business/energy
19 https://www.reuters.com/business/environment
20 https://www.reuters.com/business/finance
21 https://www.reuters.com/business/media-telecom
22 https://www.reuters.com/business/healthcare-pharmaceuticals
23 https://www.reuters.com/business/autos-transportation
24 https://ww

### 1.2.4. Building an index

In [9]:
indexer = Indexer()
k = 1
for c in indexer.crawl_generator_for_index("https://www.reuters.com/", 3, "docs_collection"):
    print(k, c.url)
    k += 1

1 https://www.reuters.com/
2 https://www.reuters.com/home
3 https://www.reuters.com/world
4 https://www.reuters.com/world/us-politics
5 https://www.reuters.com/world/us
6 https://www.reuters.com/world/uk
7 https://www.reuters.com/world/china
8 https://www.reuters.com/world/india
9 https://www.reuters.com/world/americas
10 https://www.reuters.com/world/asia-pacific
11 https://www.reuters.com/world/europe
12 https://www.reuters.com/world/middle-east-africa
13 https://www.reuters.com/world/the-great-reboot
14 https://www.reuters.com/business
15 https://www.reuters.com/business/sustainable-business
16 https://www.reuters.com/business/legal
17 https://www.reuters.com/business/energy
18 https://www.reuters.com/business/environment
19 https://www.reuters.com/business/finance
20 https://www.reuters.com/business/media-telecom
21 https://www.reuters.com/business/healthcare-pharmaceuticals
22 https://www.reuters.com/business/autos-transportation
23 https://www.reuters.com/business/aerospace-defen

93 https://www.reuters.com/news/pictures
94 https://www.reuters.com/news/picture/protests-erupt-after-black-man-shot-dead-idUSRTXBEDJE
95 https://www.reuters.com/news/picture/photos-of-the-week-idUSRTXBCF81
96 https://www.reuters.com/finance
97 https://www.reuters.com/article/us-autos-southkorea-shares/shares-in-sk-innovation-surge-after-settlement-with-rival-brightens-u-s-prospects-idUSKBN2BZ00N
98 https://www.reuters.com/article/us-usa-sec-esg/u-s-sec-review-of-socially-responsible-funds-finds-potentially-misleading-claims-idUSKBN2BW2SZ
99 https://www.reuters.com/article/us-usa-bonds-supply/analysis-big-u-s-treasury-auctions-could-restart-rise-in-yields-idUSKBN2BZ0C0
100 https://www.reuters.com/article/us-newzealand-economy-rbnz/nzs-cenbank-to-stand-pat-as-it-assesses-travel-resumption-property-curbs-reuters-poll-idUSKBN2BZ04S
101 https://www.reuters.com/article/us-japan-economy-prices/japan-wholesale-prices-rise-for-first-time-in-13-months-as-global-recovery-boosts-commodity-costs-i

160 https://www.reuters.com/article/uk-soccer-england-tot-mun-report-wrapup/man-utd-beat-spurs-west-ham-maintain-top-four-charge-idUSKBN2BY0S3
161 https://www.reuters.com/article/uk-health-coronavirus-eu-astrazeneca/astrazeneca-says-it-had-positive-meeting-with-eu-over-vaccine-row-idUSKBN2BY0HG
162 https://www.reuters.com/news/archive/ukdomestic?view=page&page=2&pageSize=10
163 https://www.reuters.com/article/us-china-alibaba-breakingviews/breakingviews-fine-leaves-sharp-sword-hanging-over-alibaba-idUSKBN2BZ06U
164 https://www.reuters.com/article/us-china-alibaba-breakingviews/breakingviews-fine-leaves-sharp-sword-hanging-over-alibaba-idUSKBN2BZ06B
165 https://www.reuters.com/article/us-usa-taiwan/blinken-warns-of-chinas-increasingly-aggressive-actions-against-taiwan-idUSKBN2BY0GO
166 https://www.reuters.com/article/us-health-coronavirus-china-cases/mainland-china-reports-16-new-covid-19-cases-vs-10-a-day-earlier-idUSKBN2BY00T
167 https://www.reuters.com/article/us-china-tech-antitrust

219 https://www.reuters.com/article/us-britain-royals-philip-funeral/prince-philip-to-have-ceremonial-funeral-on-april-17-harry-to-attend-idUSKBN2BX0EH
220 https://www.reuters.com/article/us-ukraine-crisis-minister/ukraine-says-it-could-be-provoked-by-russian-aggression-in-conflict-area-idUSKBN2BX072
221 https://www.reuters.com/article/us-health-coronavirus-france-cases/french-coronavirus-intensive-care-cases-and-deaths-keep-rising-idUSKBN2BX0G6
222 https://www.reuters.com/article/us-britain-royals-philip-salutes/prince-philips-death-marked-by-gun-salutes-across-uk-idUSKBN2BX09E
223 https://www.reuters.com/article/us-portugal-corruption-socrates/portugal-ex-pm-socrates-to-face-trial-for-alleged-money-laundering-graft-charges-dropped-idUSKBN2BW29C
224 https://www.reuters.com/article/us-britain-nireland-loyalists/northern-irish-loyalists-demand-brexit-changes-call-for-end-to-street-violence-idUSKBN2BW20Y
225 https://www.reuters.com/news/archive/rcom-europe?view=page&page=2&pageSize=10
22

286 https://www.reuters.com/article/uk-climate-change-ecb/big-ecb-climate-change-role-may-be-step-too-far-warns-wunsch-idUSKBN2BT1QW
287 https://www.reuters.com/article/us-climate-change-central-banks/china-and-brazil-have-worlds-greenest-central-banks-activists-say-idUSKBN2BN01K
288 https://www.reuters.com/article/us-jp-morgan-esg/jpmorgan-forms-new-team-in-commercial-banking-unit-as-part-of-green-push-idUSKBN2BV28J
289 https://www.reuters.com/article/us-norilsknickel-dividend/nornickel-considers-2-billion-buyback-in-compensation-for-lower-dividend-idUSKBN2BW2UT
290 https://www.reuters.com/news/archive/esgnews?view=page&page=2&pageSize=10
291 https://www.reuters.com/news/archive/esg
292 https://www.reuters.com/article/knight-merger/holland-knight-tie-up-may-usher-in-the-next-wave-of-big-law-mergers-idUSL1N2M2372
293 https://www.reuters.com/article/lawyer-merge-holland/holland-knight-and-thompson-knight-in-merger-talks-idUSL1N2M228W
294 https://www.reuters.com/article/idUSKBN2BW22G
295

...

580 https://www.reuters.com/quote/.DJI
581 https://www.reuters.com/quote/.IXIC
582 https://www.reuters.com/quote/.NYA
583 https://www.reuters.com/quote/.SPX
584 https://www.reuters.com/quote/.GSPTSE
585 https://www.reuters.com/quote/.FTSE
586 https://www.reuters.com/quote/.STOXX50E
587 https://www.reuters.com/quote/.GDAXI
588 https://www.reuters.com/quote/.FCHI
589 https://www.reuters.com/quote/.IBEX
590 https://www.reuters.com/quote/.N225
591 https://www.reuters.com/quote/.HSI
592 https://www.reuters.com/quote/.AORD
593 https://www.reuters.com/quote/.SSEC
594 https://www.reuters.com/quote/.BSESN
595 https://www.reuters.com/quote/SPc1
596 https://www.reuters.com/quote/NQc1
597 https://www.reuters.com/quote/FFIc1
598 https://www.reuters.com/quote/FCEc1
599 https://www.reuters.com/quote/FDXc1
600 https://www.reuters.com/quote/NKc1
601 https://www.reuters.com/quote/JNIc1
602 https://www.reuters.com/article/us-britain-greensill-cameron/uk-former-pm-cameron-says-greensill-lobbying-should-ha

676 https://www.reuters.com/article/global-forex/dollar-turns-higher-as-risk-sentiment-wanes-idUSKBN2991SN
677 https://www.reuters.com/article/uk-global-forex/covid-19-proof-risk-sentiment-drags-dollar-near-2018-lows-idUSKBN29902F
678 https://www.reuters.com/article/global-forex-int/more-weakness-seen-as-dollar-posts-worst-year-since-2017-idUSKBN29501O
679 https://www.reuters.com/article/uk-britain-sterling/pound-extends-gains-on-new-post-brexit-transition-time-for-swaps-trading-idUSKBN2950XS
680 https://www.reuters.com/article/global-forex-int/dollar-plumbs-more-than-two-year-lows-as-more-stimulus-in-view-idUSKBN29402M
681 https://www.reuters.com/article/global-forex-int/dollar-pares-losses-as-senates-mcconnell-vague-on-further-stimulus-idUSKBN29307G
682 https://www.reuters.com/article/us-global-forex/euro-beats-yen-sterling-as-brexit-u-s-stimulus-boost-risk-appetite-idUSKBN292059
683 https://www.reuters.com/article/uk-global-forex/pound-off-two-and-a-half-year-peak-after-brexit-deal-

762 https://www.reuters.com/article/us-art-kpop/from-stage-to-canvas-k-pop-stars-prepare-for-london-art-exhibition-idUSKBN2BW057
763 https://www.reuters.com/article/us-nigeria-beverages-spirit/nigerian-duo-hope-to-turn-local-spirit-into-premium-brand-idUSKBN2BW187
764 https://www.reuters.com/article/us-retail-trading-nfts-new-york-gallery/digital-art-gets-physical-home-buyers-in-new-york-gallery-idUSKBN2BV2TE
765 https://www.reuters.com/article/us-egypt-antiquities/archaeologists-unearth-ancient-egyptian-pompeii-near-luxor-idUSKBN2BV2J2
766 https://www.reuters.com/article/us-fashion-armani-partner/giorgio-armani-could-consider-an-italian-partner-magazine-idUSKBN2BU2NX
767 https://www.reuters.com/article/us-germany-alpaca/orphaned-and-disabled-baby-alpaca-walks-again-with-her-own-set-of-wheels-idUSKBN2BV1MV
768 https://www.reuters.com/video/2021/04/09/taylor-swift-re-releases-hit-fearless-al?videoId=728294593&videoChannel=1004
769 https://www.reuters.com/video/2021/04/08/dionne-warwick-

833 https://www.reuters.com/article/us-sailing-americas-cup/sailing-team-new-zealand-savour-americas-cup-triumph-in-home-waters-idUSKBN2B90GK
834 https://www.reuters.com/article/us-rugby-union-nations-women/rugby-england-crush-scotland-52-10-in-womens-six-nations-opener-idUSKBN2BQ0II
835 https://www.reuters.com/article/us-rugby-union-super-au/rugby-reds-make-it-six-wins-out-six-to-top-super-rugby-au-standings-idUSKBN2BQ0AM
836 https://www.reuters.com/article/us-rugby-lgbt-sport-trfn/england-rugby-proposes-height-weight-limits-for-trans-women-idUSKBN2BN2Q4
837 https://www.reuters.com/news/archive/sport-athletics
838 https://www.reuters.com/article/olympics-2020-athletics-australia/olympics-australia-pulls-out-of-athletics-relay-championships-due-to-covid-19-idUSL4N2M50S5
839 https://www.reuters.com/article/athletics-doping-russia/athletics-russian-former-indoor-1500-metres-champion-soboleva-gets-eight-year-doping-ban-idUSL1N2M01ER
840 https://www.reuters.com/article/us-athletics-shoes/a

898 https://www.reuters.com/news/pictures/archive?page=2&pageSize=9
899 https://www.reuters.com/video/2021/03/31/images-of-march?videoId=727951398&videoChannel=118069
900 https://www.reuters.com/video/2021/03/01/images-of-february?videoId=726653738&videoChannel=118069
901 https://www.reuters.com/video/2021/02/01/images-of-january?videoId=725142268&videoChannel=118069
902 https://www.reuters.com/video/breakingviews
903 https://www.reuters.com/video/business
904 https://www.reuters.com/video/marketsnow
905 https://www.reuters.com/video/great-reboot
906 https://www.reuters.com/video/change-suite
907 https://www.reuters.com/video/sectors-upclose
908 https://www.reuters.com/video?feat=1WgdzxmbJQ4M
909 https://www.reuters.com/video/technology
910 https://www.reuters.com/video/world
911 https://www.reuters.com/video/entertainment
912 https://www.reuters.com/video/politics
913 https://www.reuters.com/video/oddly-enough
914 https://www.reuters.com/news/archive/healthPharmaNews
915 https://www.r

### 1.2.5. [5] Index statistics

Load an index and print the statistics.

In [10]:
# load index, doc_lengths and doc_urls
with open('inverted_index.p', 'rb') as fp:
    index = pickle.load(fp)
with open('doc_lengths.p', 'rb') as fp:
    doc_lengths = pickle.load(fp)
with open('doc_urls.p', 'rb') as fp:
    doc_urls = pickle.load(fp)
    
    
print('Total index length', len(index))
print('\nTop terms by number of documents they apperared in:')
sorted_by_n_docs = sorted(index.items(), key=lambda kv: (len(kv[1]), kv[0]), reverse=True)
print([(sorted_by_n_docs[i][0], len(sorted_by_n_docs[i][1])) for i in range(20)])
print('\nTop terms by overall frequency:')
sorted_by_freq = sorted(index.items(), key=lambda kv: (kv[1][0], kv[0]), reverse=True)
print([(sorted_by_freq[i][0], sorted_by_freq[i][1][0]) for i in range(20)])

Total index length 9980

Top terms by number of documents they apperared in:
[('reuter', 490), ('s', 480), ('said', 434), ('ha', 377), ('wa', 366), ('have', 362), ('after', 310), ('thi', 303), ('more', 299), ('which', 298), ('year', 297), ('not', 297), ('but', 285), ('last', 260), ('had', 258), ('up', 254), ('also', 254), ('their', 245), ('than', 243), ('new', 243)]

Top terms by overall frequency:
[('s', 2465), ('said', 1594), ('wa', 1081), ('ha', 944), ('have', 905), ('reuter', 674), ('not', 674), ('more', 639), ('year', 636), ('but', 571), ('thi', 570), ('which', 566), ('after', 562), ('had', 529), ('new', 503), ('we', 500), ('who', 497), ('compani', 491), ('their', 484), ('i', 478)]


## 1.3. [15] Answering a query (finally)

Now, given that we already have built the inverted index, it's time to utilize it for answering user queries. In this class there are two methods you need to implement:
- `boolean_retrieval`, the simplest form of document retrieval which returns a set of documents such that each one contains all query terms. Returns a set of document ids. Refer to *ch.1* of the book for details;
- `okapi_scoring`, Okapi BM25 ranking function - assigns scores to documents in the collection that are relevant to the user query. Returns a dictionary of scores, `doc_id:score`. Read about it in [Wikipedia](https://en.wikipedia.org/wiki/Okapi_BM25#The_ranking_function) and implement accordingly.

Both methods accept `query` parameter in a form of a dictionary, `term:frequency`

In [11]:
from collections import Counter
import math

class QueryProcessing:
    
    @staticmethod
    def prepare_query(raw_query):
        prep = Preprocessor()
        # pre-process query the same way as documents
        query = prep.preprocess(raw_query)
        # count frequency
        return Counter(query)
    
    @staticmethod
    def boolean_retrieval(query, index):
        #TODO retrieve a set of documents containing all query terms
        query_items = list(map(lambda x: x[0], query.items()))
    
        relev_docs = set(list(map(lambda x: x[0], index[query_items[0]][1:])))
        for term in query_items:
            relev_docs = relev_docs & set(list(map(lambda x: x[0], index[term][1:])))
        return relev_docs

    
    @staticmethod
    def okapi_scoring(query, doc_lengths, index, k1=1.2, b=0.75):

        N = len(doc_lengths)
        avg_length = sum(list(doc_lengths.values())) / N
        okapi = {}

        for doc_id, doc_length in doc_lengths.items():
            okapi_score = 0
            
            for term in query:
                n_q = len(index[term][1:])
                idf = math.log((N - n_q + 0.5) / (n_q + 0.5) + 1 )
                doc_ids = list(map(lambda x: x[0], index[term][1:]))
                if doc_id in doc_ids:
                    idx = doc_ids.index(doc_id)
                    term_freq = index[term][idx+1][1]
                else:
                    term_freq = 0
                koef = (term_freq * (k1 + 1)) / (term_freq + k1 * (1 - b + b * doc_length / avg_length))
                okapi_score += idf * koef

            if okapi_score != 0:
                okapi[doc_id] = okapi_score
            
        return okapi

### 1.3.1. Tests

In [12]:
test_doc_lengths = {1: 20, 2: 15, 3: 10, 4:20, 5:30}
test_index = {'x': [2, (1, 1), (2, 1)], 'y': [2, (1, 1), (3, 1)], 'z': [3, (2, 1), (4,2)]}

test_query1 = QueryProcessing.prepare_query('x z')
test_query2 = QueryProcessing.prepare_query('x y')


assert QueryProcessing.boolean_retrieval(test_query1, test_index) == {2}
assert QueryProcessing.boolean_retrieval(test_query2, test_index) == {1}
okapi_res = QueryProcessing.okapi_scoring(test_query2, test_doc_lengths, test_index)
assert all(k in okapi_res for k in (1, 2, 3))
assert not any(k in okapi_res for k in (4, 5))
assert okapi_res[1] > okapi_res[3] > okapi_res[2]
okapi_res

{1: 1.7140324693860898, 2: 0.9579736445390843, 3: 1.0858929739285763}

# 2. [40] Sorri not veri gud in inglish

Have you ever googled someone's name without knowing exactly how it should be written? Were you ever reluctant to look up the correct spelling of a query you typed? Or just unable to type properly because of being in a rush? Modern search engines usually do a pretty good job in deciphering defective user input. In order to be able to do that, a good spell-checking mechanism should be incorporated into a search procedure. Today we will take one step further towards building a good search engine and work on tolerant retrieval with respect to user queries. We will consider two cases:

1. Users know that they don't know the correct spelling OR they want to get the results that follow some known pattern, so the use so called wildcards - queries like `retr*val`;
2. Users don't know the correct spelling OR they don't care OR they are in a rush OR they expect that mistakes will be corrected OR /your option/... so they make mistakes and we need to handle them using:

    2.1. Trigrams with Jaccard coefficient;
    
    2.2. Simple spellchecker by Peter Norvig with QWERTY weights;

## 2.1. [25] Handling wildcards

We will handle wildcard queries using k-grams. K-gram is a list of consecutive k chars in a string - i.e., for the word *'star'*, it will be '*\$st*', '*sta*', '*tar*', and '*ar$*', if we take k=3. Take a look at [book](https://nlp.stanford.edu/IR-book/pdf/irbookonlinereading.pdf) *chapter 3.2.2* to understand how k-grams can help efficiently match a wildcard against dictionary words. Here we will only consider wildcards with star symbols (may be multiple).

Notice that for building k-grams index, **we will need a vocabulary of original word forms** to compare words in user input to the vocabulary of "correct" words (think why inverted index which we built for stemmed words doesn't work here).   

You need to implement the following:

- `build_inverted_index_orig_forms` - creates inverted index of original world forms from `facts` list, which is already given to you.  
    Output format: `term:[collection_frequency, (doc_id_1, doc_freq_1), (doc_id_2, doc_freq_2), ...]`
    

- `build_k_gram_index` - creates k-gram index which maps every k-gram encountered in facts collection to a list of words containing this k-gram. Use the abovementioned inverted index of original words to construct this index.  
    Output format: `'k_gram': ['word1_with_k_gram', 'word2_with_k_gram', ...]`
    
    
- `generate_wildcard_options` - produce a list of vocabulary words matching given wildcard by intersecting postings of k-grams present in the wildcard (refer to *ch 3.2.2*). 

- `search_wildcard` - return list of facts that contain the words matching a wildcard query.


We will use the dataset with curious facts for testing.

### 2.1.1. Downloading the dataset

In [13]:
import urllib.request
data_url = "https://raw.githubusercontent.com/IUCVLab/information-retrieval/main/datasets/facts.txt"
local_filename, headers = urllib.request.urlretrieve(data_url)

facts = []
with open(local_filename) as fp:
    for cnt, line in enumerate(fp):
        facts.append(line.strip('\n'))
        
print(*facts[-5:], sep='\n')

151. Women have twice as many pain receptors on their body than men. But a much higher pain tolerance.
152. There are more stars in space than there are grains of sand on every beach in the world.
153. For every human on Earth there are 1.6 million ants.
154. The total weight of all those ants, however, is about the same as all the humans.
155. On Jupiter and Saturn it rains diamonds.


### 2.1.2. [25] Implementation of search

In [14]:
import re


def build_inverted_index_orig_forms(documents):
    #TODO build an inverted index of original word forms 
    # (without stemming, just word tokenized and lowercased)
    inverted_index = {}
    prep = Preprocessor()
    
    for i, doc in enumerate(documents):
        words = prep.tokenize(doc.lower())
        words = list(filter(prep.is_apt_word, words))
        doc_id = i
        
        for word in words:
            if word in inverted_index:
                inverted_index[word][0] += 1
                doc_ids = list(map(lambda x: x[0], inverted_index[word][1:]))

                if doc_id in doc_ids:
                    ind = doc_ids.index(doc_id)
                    count = inverted_index[word][ind+1][1]
                    inverted_index[word][ind+1] = (doc_id, count+1)
                else:
                    inverted_index[word].append((doc_id, 1))
            else:
                inverted_index[word] = []
                inverted_index[word].append(1)
                inverted_index[word].append((doc_id, 1))
   
    return inverted_index


def build_k_gram_index(inverted_index, k):
    #TODO build index of k-grams for dictionary words. 
    # Padd with '$' ($word$) before splitting to k-grams    
    k_gram_index = {}
    
    for word in inverted_index:
        padded_word = "$" + word + "$"
        for i in range(len(padded_word) - k + 1):
            k_gram = padded_word[i:i+k]

            if k_gram not in k_gram_index:
                k_gram_index[k_gram] = []
                k_gram_index[k_gram].append(word)
            else:
                if word not in k_gram_index[k_gram]:
                    k_gram_index[k_gram].append(word)
    
    return k_gram_index


def generate_wildcard_options(wildcard, k_gram_index, inverted_index):
    #TODO for a given wildcard return all words matching it using k-grams
    # refer to book chapter 3.2.2
    # don't forget to pad wildcard with '$', when appropriate
    padded_wildcard = "$" + wildcard + "$"
    k = len(list(k_gram_index.items())[0][0])
    words = set(inverted_index.keys())
    
    for part in padded_wildcard.split("*"):
        for i in range(len(part) - k + 1):
            if part[i:i+k] in k_gram_index:
                words = words & set(k_gram_index[part[i:i+k]])
            else: 
                return []
    words = list(words)
    # a post-filtering step
    wildcard = wildcard.replace("*", ".+")
    
    return [w for w in words if re.search(wildcard, w)]


def search_wildcard(wildcard, k_gram_index, index, docs):
    #TODO retrive list of documnets (facts) that contain words matching wildcard
    words = generate_wildcard_options(wildcard, k_gram_index, index)
    resulted_docs = set()
    
    for w in words:
        for doc_id in list(map(lambda x: x[0], index[w][1:])):
            resulted_docs.add(docs[doc_id])
    return list(resulted_docs)

### 2.1.3. Tests

In [15]:
index_orig_forms = build_inverted_index_orig_forms(facts)
k_gram_index = build_k_gram_index(index_orig_forms, 3)

wildcard = "re*ed"

wildcard_options = generate_wildcard_options(wildcard, k_gram_index, index_orig_forms)
print(wildcard_options)
assert(len(wildcard_options) >= 3)
assert("red" not in wildcard_options) 

wildcard_results = search_wildcard(wildcard, k_gram_index, index_orig_forms, facts)
# some pretty printing
for r in wildcard_results:
    # highlight terms for visual evaluation
    for term in wildcard_options:
        r = re.sub(r'(' + term + ')', r'\033[1m\033[91m\1\033[0m', r, flags=re.I)
    print(r)

assert(len(wildcard_results) >=3)

assert "13. James Buchanan, the 15th U.S. president continuously bought slaves with his own money in order to free them." in search_wildcard("pres*dent", k_gram_index, index_orig_forms, facts)
assert "40. 9 out of 10 Americans are deficient in Potassium." in search_wildcard("p*tas*um", k_gram_index, index_orig_forms, facts)
assert "61. A man from Britain changed his name to Tim Pppppppppprice to make it harder for telemarketers to pronounce." in search_wildcard("*price", k_gram_index, index_orig_forms, facts)

['received', 'recorded', 'reduced']
134. A person can live without food for about a month, but only about a week without water. If the amount of water in your body is reduced by just 1%, you'll feel thirsty. If it's reduced by 10%, you'll die.
4. The largest recorded snowflake was in Keogh, MT during year 1887, and was 15 inches wide.
102. More than 50% of the people in the world have never made or received a telephone call.


## 2.2. [15] Handling typos

### 2.2.0. Dataset 

Download github typo dataset from [here](https://github.com/mhagiwara/github-typo-corpus).
Load it with this code:

In [16]:
!pip install jsonlines

You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [17]:
import jsonlines

dataset_file = "github-typo-corpus.v1.0.0.jsonl"

dataset = []
other_langs = set()

with jsonlines.open(dataset_file) as reader:
    for obj in reader:
        for edit in obj['edits']:
            if edit['src']['lang'] != 'eng':
                other_langs.add(edit['src']['lang'])
                continue

            if edit['is_typo']:
                src, tgt = edit['src']['text'], edit['tgt']['text']
                if src.lower() != tgt.lower():
                    dataset.append((edit['src']['text'], edit['tgt']['text']))
                
print(f"Dataset size = {len(dataset)}")

Dataset size = 245909


#### Explore sample typos
Please, explore the dataset. You may see, that this is
- mostly markdown
- some common mistakes with do/does
- some just refer to punctuation typos (which we do not consider)

In [18]:
for pair in dataset[1010:1020]:
    print(f"{pair[0]} => {pair[1]}")

        """Make am instance. =>         """Make an instance.
* travis: test agains Node.js 11 => * travis: test against Node.js 11
The parser receive a string and returns an array inside a user-provided  => The parser receives a string and returns an array inside a user-provided 
CSV data is send through the `write` function and the resulted data is obtained => CSV data is sent through the `write` function and the resulting data is obtained
One useful function part of the Stream API is `pipe` to interact between  => One useful function of the Stream API is `pipe` to interact between 
source to a `stream.Writable` object destination. This example available as  => source to a `stream.Writable` object destination. This example is available as 
`node samples/pipe.js` read the file, parse its content and transform it. => `node samples/pipe.js` and reads the file, parses its content and transforms it.
Most of the generator is imported from its parent project [CSV][csv] in a effort  => Most o

#### 2.2.0.1. Build a dataset vocabulary
Here we prepare a vocabulary for spellchecker testing and for estimating overall correction quality. Consider only word-level. Be carefull, there is markdown (e.g. \`name\`. \[url\]\(http://url)) and comment symbols (\#, //, \*).

In [19]:
def sent_to_words(sent):
    # splits sentence to words, filtering out non-alphabetical terms
    words = nltk.word_tokenize(sent)    
    words_filtered = filter(lambda x: x.isalpha(), words)
    return words_filtered

In [20]:
vocabulary = Counter()
for pair in dataset:
    for word in sent_to_words(pair[1].lower()):
        vocabulary[word] += 1
len(vocabulary)

63724

In [21]:
from itertools import islice
print(list(islice(vocabulary.items(), 10)))

[('function', 6193), ('de', 82), ('deutsch', 4), ('nocomments', 2), ('you', 42075), ('can', 26027), ('disable', 532), ('comments', 360), ('for', 44756), ('the', 207017)]


### 2.2.1. [15] Implement context-independent spellcheker (Trigrams with Jaccard coefficient) ##

In [22]:
def fix_typo_kgram(word, k_gram_index) -> list:
    # k = 3
    k = len(list(k_gram_index.keys())[0])
    k_grams = []
    
    padded_word = "$" + word + "$"
    for i in range(len(padded_word) - k + 1):
        k_grams.append(padded_word[i:i+k])
    
    words = []
    
    for k_gram in k_gram_index:
        if k_gram in k_grams:
            words.append(k_gram_index[k_gram])
            
    approp_words = set()

    for w in words:
        approp_words = approp_words | set(w)
    
    top_words = []
    
    for w in approp_words:
        w_k_grams = []
        padded_w = "$" + w + "$"
        
        for i in range(len(padded_w) - k + 1):
            w_k_grams.append(padded_w[i:i+k])

        jaccard_coef = len(set(w_k_grams) & set(k_grams)) / len(set(w_k_grams) | set(k_grams))
        
        if jaccard_coef > 0.3:
            top_words.append(w)
            
    return top_words

### 2.2.2. Tests

In [23]:
# tests

k_gram_index_github = build_k_gram_index(vocabulary, 3)
print(fix_typo_kgram("enouh", k_gram_index_github)[:20])
assert "enough" in fix_typo_kgram("enouh", k_gram_index_github), "Assert k-gram failed"

['enough', 'enought', 'eno']


## 2.3. [Bonus]

### 2.3.1. QWERTY - Editorial distance

Write code to compute weighted QWERTY-editorial distance.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/da/KB_United_States.svg/640px-KB_United_States.svg.png" width="640"/> 

Use this image to prepare weight function:
- all letter pairs which share vertical border will get 0.5 multiplier **for replace** (`df`, `cv`, `ui`, ...)
- all letter pairs which share at least some horizontal border will get 0.7 multiplier **for replace** (`dc`, `dr`, `km`, ...)
- other operations are not scaled (x1 multiplier).

In [24]:
def replace_weight(let1, let2) -> float:
    # TODO what is the weight for a pair of letters being replaced? Note this function should be commutative
    horizontal  = [" ", "q", 'w', 'e', 'r', 't', 'y', 'u', 'y', 'u', 'i', 'o', 'p', ' ', 'a', 's', 'd', 'f', 'g', 'h', \
                   'j', 'k', 'l', ' ', 'z', 'x', 'c', 'v', 'b', 'n', 'm', " "]
    
    vertical = {"q": ["a"], "w": ["a", 's'], 'e': ['s', 'd'], 'r': ['d', 'f'], 
                't': ['f', 'g'], 'y': ['g', 'h'], 'u': ["h", 'j'], 'i': ["j", "k"], 'o': ['k', 'l'], 
                'p': ['l'], "a": ["q", "w", "z"], "s": ["e", "w", "z", "x"], "d": ["e", "r", "c", "x"], 
                "f": ["t", "r", "c", "v"], "g": ["t", "y", "b", "v"], "h": ["u", "y", "b", "n"], 
                "j": ["u", "i", "b", "m"], "k": ["o", "i", "m"], "l": ["o", "p"], "z": ["a", "s"], 
                "x": ["d", "s"], "c": ["d", "f"], "v": ["g", "f"], "b": ["g", "h"], "n": ["j", "h"], 
                "m": ["j", "k"]
               }
    if let1 in vertical[let2]:
        return 0.7
    elif horizontal[horizontal.index(let1)-1] == let2 or  horizontal[horizontal.index(let1)+1] == let2:
        return 0.5
    else:
        return 1

# read about Damerau-Levenshtein algorithm here 
# https://www.guyrutenberg.com/2008/12/15/damerau-levenshtein-distance-in-python/, so the code could be similar, as
# it was my reference
def qwerty_edit_dist(s1, s2):
    d = {}
    n = len(s1)
    m = len(s2)
    
    for i in range(-1, n + 1):
        d[(i, -1)] = i + 1
        
    for j in range(-1, m + 1):
        d[(-1, j)] = j + 1
 
    for i in range(n):
        
        for j in range(m):
            
            if s1[i] == s2[j]:
                replace_cost = 0
                
            else:
                replace_cost = replace_weight(s1[i], s2[j])
                
            d[(i,j)] = min(
                           d[(i-1,j)] + 1, 
                           d[(i,j-1)] + 1, 
                           d[(i-1,j-1)] + replace_cost, 
                          )
            if i > 0 and j > 0 and s1[i]==s2[j-1] and s1[i-1] == s2[j]:
                d[(i,j)] = min (d[(i,j)], d[i-2,j-2] + replace_cost) 
 
    return d[n-1, m-1]
    


In [25]:
# tests

assert qwerty_edit_dist("korrectud", "corrected") == 2.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("soem", "some") == 1.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("one", "one") == 0.0, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fihure", "figure") == 0.5, "Edit distance is computed incorrectly"
assert qwerty_edit_dist("fivure", "figure") == 0.7, "Edit distance is computed incorrectly"

### 2.3.2. Norvig's spellchecker with QWERTY weights

You can base your code on [Norvig's corrector](https://norvig.com/spell-correct.html), but you should be sure you account the fact, that typos for close letters cost less. This should be considered in ranking.

In [26]:
def fix_typo_qwerty_norvig(word) -> str:
    #TODO return best matching result for the word
    
    return ""

In [ ]:
# tests

assert fix_typo_qwerty_norvig("korrectud") == "corrected", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("speling") == "spelling", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("condidence") == "confidence", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fpx") == "fox", "Norvig's correcter doesn't work"
assert fix_typo_qwerty_norvig("fux") == "fix", "Norvig's correcter doesn't work"

### 2.3.3. Estimate quality of functions

In [ ]:
norvig, kgram = 0, 0
limit = 10000
counter = limit
for i, (src, target) in enumerate(dataset):
    if i == limit:
        break
    words = sent_to_words(src.lower())
    # word suspected for typos
    sn, sk = src.lower(), src.lower()
    for word in words:
        if word not in vocabulary and word.isalpha():
            # top-1 accuracy
            wn, wk = fix_typo_qwerty_norvig(word), \
                     fix_typo_kgram(word, k_gram_index_github)[0]
            sn = sn.replace(word, wn)
            sk = sk.replace(word, wk)
    norvig += int(sn == target.lower())
    kgram += int(sk == target.lower())

print(f"Norvig accuracy ({norvig}) = {norvig / limit}")
print(f"k-gram accuracy ({kgram}) = {kgram / limit}")